In [ ]:
import torch
import torch.nn as nn
import torchvision
from vggnet import VGGNet
from cifar10_dataloader import train_data_loader, test_data_loader

In [ ]:
# 能用 GPU 就用
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(device)

In [ ]:
epoch_num = 200
lr = 0.01
batch_size = 128

In [ ]:
net = VGGNet().to(device)

# loss
loss_func = nn.CrossEntropyLoss()

# 优化器
optimizer = torch.optim.Adam(params=net.parameters(), lr=lr)

# 动态调整学习率
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 
                                    step_size=5,# 5个 epoch 调整一次学习率
                                    gamma=0.9)

In [ ]:
for epoch in range(epoch_num):
    print(f"epoch is {epoch}")
    net.train()

    for i, data in enumerate(train_data_loader):
        # 取数据
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        # 训练网络
        outputs = net(inputs)
        loss = loss_func(outputs, labels)

        # 参数初始化
        optimizer.zero_grad()
        # 反向传播
        loss.backward()
        # 更新参数
        optimizer.step()
        # 更新学习率
        scheduler.step()

        # 记录预测对的
        _, pred = torch.max(outputs.data, dim=1)
        correct = pred.eq(labels.data).cpu().sum()

        print(f"step is {i}, loss is {loss.item()}, corr is {100.0 * correct / batch_size}%")